In [1]:
from datasets import load_dataset
from unisim import TextSim
from tabulate import tabulate
import pandas as pd

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded backend
Using tf with gpu


## Load Dataset

For this example, we use a product matching dataset 

https://huggingface.co/datasets/RUC-DataLab/ER-dataset

https://github.com/ruc-datalab/DADER

There are additional datasets that test entity retrieval and matching

citation, product matching, restaurant matching

In [43]:
dataset = load_dataset("RUC-DataLab/ER-dataset", data_files="dblp_scholar.csv", split="train")

NameError: name 'text_sim' is not defined

In [41]:
# restaurants, products are good to do like differences in matching addresses etc easy to do 

# restaurants1 not bad, better than fodors zagats which gets 100 percent
# beer is pretty good
# walmart amazon not bad
# itunes_amazon is meh maybe too easy
# dblp_sholcar and acm decent might be too easy , scholar is harder than ACM which is like 99% accuracy
# restauratnts 3 and 4 are decent but might be too easy
# fodors zagats too aesy

# not good because book editions books 4 > book 2
# anime not easy either its like the diference is saesons 
# cosmetics not good either, it's like different colors are not clasified as the same thing
# abt_buy not that good, missing descriptions and not super clean it seems 
# movies1 not good year range is kinda weird/ 
# shoes and comuters not great, there are near-dups that are marked as not the same item 
# ebboks not good, based on similarity description similarity

In [44]:
dataset[:5]

{'A_title': ['the demarcation protocol : a technique for maintaining constraints in distributed database systems ',
  'on-demand data elevation in hierarchical multimedia storage servers ',
  'database tuning : principles , experiments , and troubleshooting techniques ',
  'dna-miner : a system prototype for mining dna sequences ',
  'communication efficient distributed mining of association rules '],
 'A_authors': ['d barbarс , h garcia-molina ',
  'p triantafillou , t papadakis ',
  'd shasha , p bonnet ',
  'j han , h jamil , y lu , l chen , y liao , j pei ',
  'a schuster , r wolff '],
 'A_venue': ['vldb j. ',
  'vldb ',
  'vldb ',
  'sigmod conference ',
  'sigmod conference '],
 'A_year': [1994, 1997, 2002, 2001, 2001],
 'B_title': ['local verification of global integrity constraints in distributed databases ',
  'on-demand data elevation in a hierarchical multimedia storage server ',
  'database tuning : principles , experiments , and troubleshooting techniques ( part i ) ',
  '

In [3]:
print("Size of dataset:", len(dataset))

dataset_features = list(dataset.features.keys())
print("Dataset features:", dataset_features)

Size of dataset: 450
Dataset features: ['A_NAME', 'A_PHONENUMBER', 'A_ADDRESS', 'B_NAME', 'B_PHONENUMBER', 'B_ADDRESS', 'label']


In [4]:
text1_features = [x for x in dataset_features if x.startswith("A")]
text2_features = [x for x in dataset_features if x.startswith("B")]
is_match_feature = "label"

def get_text_pair(idx):
    ex = dataset[idx]

    text1 = " ".join(ex[x] for x in text1_features)
    text2 = " ".join(ex[x] for x in text2_features)

    label = ex[is_match_feature]
    return [text1, text2, label]

### TextSim

In [4]:
text_sim = TextSim(index_type="exact")
text_sim.similarity("this is a text", "apples") 

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


INFO: UniSim is storing a copy of the indexed data
INFO: If you are using large data corpus, consider disabling this behavior using store_data=False


0.4220390021800995

### Computing Similarity between Texts

In [8]:
text_sim.similarity("this is an example text", "This is an example txt! 😀 ")

0.9418152570724487

In [45]:
text_sim.similarity("hello", "h3110")

0.5612893104553223

In [7]:
queries = ["apple", "appl", "icecream", "house", "random"]
targets = ["apple", "ice cream", "mouse"]

results_df = text_sim.match(queries, targets, similarity_threshold=0.9)
print(results_df.head())

      query     target  similarity  is_match
0     apple      apple    1.000000      True
1      appl      apple    0.914230      True
2  icecream  ice cream    0.950734      True
3     house      mouse    0.760066     False
4    random      mouse    0.456315     False


In [10]:
# You can directly compute text embeddings using TextSim's `embed()` method
example_texts = [
    "This is an example text!",
    "You can even use extremely long texts, TextSim is capable of handling and matching arbitrarily-long texts."
]
embeddings = text_sim.embed(example_texts)
embeddings.shape  # (2, 256) dimension

(2, 256)

In [31]:
# you can maintain an index of texts as well. 
# we support Approximate Nearest Neighbor search, with index_type="approx" using USearch, which will scale to millions or even billions of indexed examples

text_sim = TextSim()

# dataset
index_examples = [
    "I love ice cream and cookies",
    "Ice cream is super delicious",
    "my mom makes the best homemade cookies 🍪🍪🍪",
    "UniSim supports very long texts as well.",
    "UniSim supports multilingual texts too. 你好!",
]
text_sim.add(index_examples)

# you want to find nearest things in your index
query_examples = [
    "I luv ice cream and cookies🍦🍪",
    "This is an example query text.",
    "Unrelated text with no match in the dataset..."
]

result_collection = text_sim.search(query_examples, similarity_threshold=0.9, k=5)

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


INFO: UniSim is storing a copy of the indexed data
INFO: If you are using large data corpus, consider disabling this behavior using store_data=False


In [32]:
text_sim.visualize(result_collection.results[0])


Query 0: "I luv ice cream and cookies🍦🍪"
Most similar matches:

  idx  is_match      similarity  text
-----  ----------  ------------  -------------------------------------------------
    3  True                0.91  I love ice cream and cookies
    4  False               0.66  Ice cream is super delicious
    5  False               0.53  my mom makes the best homemade cookies 🍪🍪🍪
    0  False               0.42  This is an example text.
    2  False               0.36  UniSim supports multilingual texts as well. 你好!


In [16]:
print(result_collection.total_matches)
results = result_collection.results

1


In [20]:
result_collection = text_sim.search(["This is some text??"], similarity_threshold=0.9, k=5)

In [21]:

text_sim.visualize(result_collection.results[0])

# text_sim.visualize(results[1])

Query 0: "This is some text??"
Most similar matches:

  idx  is_match      similarity  text
-----  ----------  ------------  -------------------------------------------
    1  True                0.95  This is some text
    2  False               0.5   These are example texts you are looking for
    0  False               0.3   asdfsdf


In [23]:
example_data = [get_text_pair(idx) for idx in range(0, 5)]

for i in range(len(example_data)):
    text1, text2, is_match = example_data[i]
    similarity = text_sim.similarity(text1, text2)
    example_data[i].append(similarity)

print(tabulate(example_data, headers=["text1", "text2", "is_match", "similarity"]))

text1                                                                  text2                                                                        is_match    similarity
---------------------------------------------------------------------  -------------------------------------------------------------------------  ----------  ------------
15 Romolo (415) 398-1359 15 Romolo Place, San Francisco, CA            15 Romolo (415) 398-1359 15 Romolo Pl, San Francisco, CA 94133                      1      0.971069
456 Shanghai Cuisine 1261 69 Mott Street, New York, NY                 Shanghai Asian Manor (212) 766-6311 21 Mott St, New York, NY 10013                  0      0.779613
5A5 Steak Lounge (415) 989-2539 244 Jackson Street, San Francisco, CA  Delicious Dim Sum (415) 781-0721 752 Jackson St, San Francisco, CA 94133            0      0.687863
9th Street Pizza (213) 627-7798 231 E 9th St, Los Angeles, CA          Han Bat Sul Lung Tang (213) 383-9499 4163 W 5th St, Los Angeles, CA 90020 

In [24]:
targets = list(set([get_text_pair(idx)[0] for idx in range(0, len(dataset))]))  # TODO does not preserve order 
print("\n".join([t for t in targets[:5]]))

Nana (312) 929-2486 3267 S. Halsted Street, Chicago, IL
Allende Restaurant (773) 477-7114 2408 N. Lincoln Avenue, Chicago, IL
Jordan's Big Ten Pub (608) 251-6375 1330 Regent St, Madison, WI
La Salsa Fresh Mexican Grill (310) 234-8338 10800 W Pico Blvd, Los Angeles, CA
Jimmy John's (608) 255-0301 527 State St, Madison, WI


In [25]:
queries = list(set([get_text_pair(idx)[1] for idx in range(0, len(dataset))]))
print("\n".join([t for t in queries[:5]]))

Colibri Mexican Bistro (415) 440-2737 438 Geary St, San Francisco, CA 94102
Chipotle Mexican Grill (608) 233-4305 4000 University Ave, Madison, WI 53705
Shallots Bistro (847) 677-3463 7016 Carpenter Rd, Skokie, IL 60076
The Heights Deli & Bottle Shop (323) 223-0708 2927 N Broadway, Los Angeles, CA 90031
La Cocina Boricua de la Familia Galarza (773) 235-7377 2420 W Fullerton Ave, Chicago, IL 60647


In [26]:
results_df = text_sim.match(queries, targets)


#  TODO find the ones that do have matches in the dataset and look them up
# TODO find a few that do not have matches and look them up --- will show that they have no match

with pd.option_context('display.max_colwidth', None):
    display(results_df.head(10))

,query,target,similarity,is_match
0,"Colibri Mexican Bistro (415) 440-2737 438 Geary St, San Francisco, CA 94102","Colibri Mexican Bistro (415) 440-2737 438 Geary Street, San Francisco, CA",0.954280,True
1,"Chipotle Mexican Grill (608) 233-4305 4000 University Ave, Madison, WI 53705","Chipotle Mexican Grill (608) 233-4305 4000 University Ave, Madison, WI",0.980320,True
2,"Shallots Bistro (847) 677-3463 7016 Carpenter Rd, Skokie, IL 60076","Shallots Bistro (847) 677-3463 7016 Carpenter Road, Skokie, IL",0.959569,True
3,"The Heights Deli & Bottle Shop (323) 223-0708 2927 N Broadway, Los Angeles, CA 90031","Yang Chow (213) 625-0811 819 N Broadway, Los Angeles, CA",0.715259,False
4,"La Cocina Boricua de la Familia Galarza (773) 235-7377 2420 W Fullerton Ave, Chicago, IL 60647","El Rinconcito Cubano (773) 489-4440 3248 W. Fullerton Avenue, Chicago, IL",0.674455,False
5,"Chang Jiang (608) 288-1888 2935 S Fish Hatchery Rd, Fitchburg, WI 53711","Rolling Pin Bake Shop (608) 270-9611 2935 S Fish Hatchery Rd, Fitchburg, WI",0.735949,False
6,"Tex Tubbs Taco Palace (608) 242-1800 2009 Atwood Ave, Madison, WI 53704","Cafe Zoma (608) 246-2009 2326 Atwood Ave, Madison, WI",0.748568,False
7,"Pho Nam Noodle House (608) 836-7040 610 Junction Rd, Madison, WI 53717","Pho Nam (608) 836-7040 610 Junction Rd Suite 109, Madison, WI",0.850172,False
8,"Pork & Mindys (844) 573-6737 1623 N Milwaukee Ave, Chicago, IL 60647","Harold's Chicken Shack (773) 252-2424 1361 N. Milwaukee Avenue, Chicago, IL",0.755033,False
9,"Hong Kong Lounge II (415) 668-8802 3300 Geary Blvd, San Francisco, CA 94118","Men Oh (415) 386-8802 5120 Geary Boulevard, San Francisco, CA",0.731154,False


### Indexing and Querying Similar Texts

In [27]:
text_sim.reset_index()

# add list of texts to the index
text_sim.add(targets)

In [28]:
result_collection = text_sim.search(queries, similarity_threshold=0.9, k=5)

In [30]:
text_sim.visualize(result_collection.results[0])

Query 0: "Colibri Mexican Bistro (415) 440-2737 438 Geary St, San Francisco, CA 94102"
Most similar matches:

  idx  is_match      similarity  text
-----  ----------  ------------  ----------------------------------------------------------------
   72  True                0.95  Colibri Mexican Bistro (415) 440-2737 438 Geary Street, San Fran
   30  False               0.75  Tommy's Mexican (415) 387-4747 5929 Geary Boulevard, San Francis
  127  False               0.74  El Metate (415) 641-7209 2406 Bryant Street, San Francisco, CA
  116  False               0.73  The Buena Vista (415) 474-5044 2765 Hyde Street, San Francisco,
   55  False               0.72  Sushi Boat (415) 781-5111 389 Geary Street, San Francisco, CA


In [31]:
new_examples = ["Googleplex (650) 253-0000 1600 Amphitheatre Parkway, Mountain View, CA 94043"]
text_sim.add(new_examples)

In [32]:
result_collection = text_sim.search(["googleplx 650-253-0000 1600 amphitheatre parkway, mountain view, ca 94043"])
result = result_collection.results[0]

text_sim.visualize(result)

Query 0: "googleplx 650-253-0000 1600 amphitheatre parkway, mountain view, ca 94043"
Most similar matches:

  idx  is_match      similarity  text
-----  ----------  ------------  ----------------------------------------------------------------
  304  True                0.92  Googleplex (650) 253-0000 1600 Amphitheatre Parkway, Mountain Vi
